# The First Stars: Formation under X-ray Feedback -- Phase Diagrams

In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import pyGadget

In [ ]:
import seaborn as sns
sns.set_style('white')
mpl.rc('font', size=20.)
mpl.rc('font', family='serif')
mpl.rc('text', usetex=True)

In [ ]:
simV = pyGadget.sim.Simulation('stampede/vanilla',length='AU', mass='solar', track_sinks=True)
sim0 = pyGadget.sim.Simulation('stampede/xr_tau2_J0',length='AU', mass='solar', track_sinks=True)
sim1 = pyGadget.sim.Simulation('stampede/xr_tau2_J1',length='AU', mass='solar', track_sinks=True)
sim2 = pyGadget.sim.Simulation('stampede/xr_tau2_J2',length='AU', mass='solar', track_sinks=True)
#sim3 = pyGadget.sim.Simulation('stampede/xr_tau2_J3',length='AU', mass='solar', track_sinks=True)

In [ ]:
#n0, n1, n2, n3, tag = 100, 102, 202, 135, '_z26'
#n0, n1, n2, n3, tag = 615, 609, 236, 277, ''
#n0, n1, n2, n3, tag = 355, 269, 201, 200, '_t0'
nV, n0, n1, n2 = 355, 255, 169, 227
#n0, n1, n2, n3, tag = 665, 575, 502, 501, '_t1k'
#n0, n1, n2, n3, tag = 974, 880, 802, 800, '_t2k'
#n0, n1, n2, n3, tag = 1128, 1032, 951, 950, '_t2500'
#n0, n1, n2, n3, tag = 1283, 1184, 1100, 1100, '_t3k'
#n0, n1, n2, n3, tag = 1437, 1337, 1251, 1250, '_t3500'
#n0, n1, n2, n3, tag = 1592, 1489, 1401, 1400, '_t4k'
#n0, n1, n2, n3, tag = 1900, 1794, 1701, 1700, '_t5k'

In [ ]:
snapV = simV.load_snapshot(nV, 'ndensity', 'temp', 'masses', 'coordinates')
snap0 = sim0.load_snapshot(n0, 'ndensity', 'temp', 'masses', 'coordinates')
snap1 = sim1.load_snapshot(n1, 'ndensity', 'temp', 'masses', 'coordinates')
snap2 = sim2.load_snapshot(n2, 'ndensity', 'temp', 'masses', 'coordinates')
#snap3 = sim3.load_snapshot(n3, 'ndensity', 'temp', 'masses', 'coordinates')
snaplist = [snapV, snap0, snap1, snap2]

In [ ]:
for snap in snaplist:
    xyz = snap.gas.get_coords()
    uvw = snap.gas.get_velocities()
    snap.update_sink_frame_ofR(xyz, uvw)
    i = 0
    #print 'Locating sink...'
    #while snap.sinks[i].pid != snap.sim.sink1.sink_id:
    #    i += 1
    #print 'Done'
    #sink = snap.sinks[i]
    #sinkpos = (sink.x, sink.y, sink.z)
    #sinkvel = (sink.vx, sink.vy, sink.vz)
    pos = snap.gas.get_coords(system='spherical', centering='avg')
                              #center=sinkpos, vcenter=sinkvel)
    snap.update_sink_frame_ofR(snap.gas.coordinates, snap.gas.velocities)

In [ ]:
r1, r2, r3, r4, r5 = 1e3, 2e3, 3e3, 4e3, 5e3
r1List = []
r2List = []
r3List = []
r4List = []
r5List = []
for snap in snaplist:
    print snap.sim.name
    inR1 = np.where(snap.gas.spherical_coords[:,0] <= r1)[0]
    print '   ', inR1.size, "particles inside", r1, snap.sim.units.length_unit
    inR2 = np.where((snap.gas.spherical_coords[:,0] > r1) & (snap.gas.spherical_coords[:,0] <= r2))[0]
    print '   ', inR2.size, "particles between", r1, 'and', r2, snap.sim.units.length_unit
    inR3 = np.where((snap.gas.spherical_coords[:,0] > r2) & (snap.gas.spherical_coords[:,0] <= r3))[0]
    print '   ', inR3.size, "particles between", r2, 'and', r3, snap.sim.units.length_unit
    inR4 = np.where((snap.gas.spherical_coords[:,0] > r3) & (snap.gas.spherical_coords[:,0] <= r4))[0]
    print '   ', inR4.size, "particles between", r3, 'and', r4, snap.sim.units.length_unit
    inR5 = np.where((snap.gas.spherical_coords[:,0] > r4) & (snap.gas.spherical_coords[:,0] <= r5))[0]
    print '   ', inR5.size, "particles between", r4, 'and', r5, snap.sim.units.length_unit
    r1List.append(inR1)
    r2List.append(inR2)
    r3List.append(inR3)
    r4List.append(inR4)
    r5List.append(inR5)
rls = [r1List, r2List, r3List, r4List, r5List]

In [ ]:
m1List = []
m2List = []
m3List = []
m4List = []
m5List = []
mls = [m1List, m2List, m3List, m4List, m5List]
for i,snap in enumerate(snaplist):
    for j, ml in enumerate(mls):
        ml.append(snap.gas.masses[rls[j][i]].sum())

In [ ]:
n1List = []
n2List = []
n3List = []
n4List = []
n5List = []
nls = [n1List, n2List, n3List, n4List, n5List]
for i,snap in enumerate(snaplist):
    for j, nl in enumerate(nls):
        nl.append(snap.gas.ndensity[rls[j][i]].mean())

In [ ]:
T1List = []
T2List = []
T3List = []
T4List = []
T5List = []
TLs = [T1List, T2List, T3List, T4List, T5List]
for i,snap in enumerate(snaplist):
    for j,TL in enumerate(TLs):
        T = pyGadget.analyze.reject_outliers(snap.gas.temp[rls[j][i]])
        TL.append(T.mean())

In [ ]:
mdata = pd.DataFrame(mls,index=['r1e3', 'r2e3', 'r3e3', 'r4e3','r5e3'], columns=['J=0', 'J0', 'J1', 'J2'])
ndata = pd.DataFrame(nls,index=['r1e3', 'r2e3', 'r3e3', 'r4e3','r5e3'], columns=['J=0', 'J0', 'J1', 'J2'])
Tdata = pd.DataFrame(TLs,index=['r1e3', 'r2e3', 'r3e3', 'r4e3','r5e3'], columns=['J=0', 'J0', 'J1', 'J2'])

In [ ]:
mdata

In [ ]:
ax = mdata.plot(grid=False, figsize=(12,9), lw=3)
ax.set_xlabel('Radius [$10^3$ AU]')
ax.set_ylabel('Mass [M$_{\odot}$]')

In [ ]:
ax = ndata.plot(grid=False, logy=True, figsize=(12,9), lw=3)
ax.set_xlabel('Radius [AU]')
ndata

In [ ]:
ax = Tdata.plot(grid=False, figsize=(12,9), lw=3)
ax.set_xlabel('Radius [AU]')
Tdata

## Density - Temperature phase diagram

In [ ]:
def plot_temp(ax, snap, selection, **kwargs):
    dens = snap.gas.get_number_density()
    temp = snap.gas.get_temperature()
    #ax.hexbin(dens[selection],temp[selection], gridsize=250, bins='log', xscale='log', yscale='log', mincnt=1)
    ax.scatter(dens[selection],temp[selection], **kwargs)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.axhline(2.725 * (snap.header.Redshift + 1),
               linestyle='--', color='k')
    return ax
    
#with sns.axes_style("white"):
fig, axarr = plt.subplots(5,4, sharex=True, sharey=True, figsize=(16,16))
for i,snap in enumerate(snaplist):
    for j, ml in enumerate(mls):
        axarr[j,i] = plot_temp(axarr[j,i], snap, rls[j][i], s=1, c='k')
        #axarr[j,i].plot(ndata.iat[j,i], Tdata.iat[j,i], 'ro', ms=8)
        axarr[j,i].text(.03,.05, '{:.2f}'.format(mdata.iat[j,i]) + r'$\,$M$_{\odot}$', 
                        ha='left', va='bottom', size=20, transform=axarr[j,i].transAxes)
        
axarr[0,0].set_xlim(1e5, 1.5e12)
axarr[0,0].set_ylim(10, 2e4)
for a in axarr[:, 0]:
    a.set_ylabel('Temperature [K]')
for a in axarr[-1, :]:
    a.set_xlabel('n [cm$^{-3}$]') 

plt.setp([a.get_xticklabels()[1::2] for a in axarr[-1, :]], visible=False)
fig.subplots_adjust(hspace=0, wspace=0)

axarr[0,0].set_title(r'J = 0')
axarr[0,1].set_title(r'J = J$_{0}$')
axarr[0,2].set_title(r'J = 10 J$_{0}$')
axarr[0,3].set_title(r'J = 100 J$_{0}$')

axarr[0,0].text(.03,.95, r'$r\leq 10^3$ AU', ha='left', va='top', size=20, transform=axarr[0,0].transAxes)
axarr[1,0].text(.03,.95, r'$1 < r \leq 2$', ha='left', va='top', size=20, transform=axarr[1,0].transAxes)
axarr[2,0].text(.03,.95, r'$2 < r \leq 3$', ha='left', va='top', size=20, transform=axarr[2,0].transAxes)
axarr[3,0].text(.03,.95, r'$3 < r \leq 4$', ha='left', va='top', size=20, transform=axarr[3,0].transAxes)
axarr[4,0].text(.03,.95, r'$4 < r \leq 5$', ha='left', va='top', size=20, transform=axarr[4,0].transAxes)

fig.savefig('figures/radial_bins/temp.png', bbox_inches='tight')

In [ ]:
def plot_temp(ax, snap, selection, **kwargs):
    dens = snap.gas.get_number_density()
    temp = snap.gas.get_temperature()
    cmb = kwargs.pop('cmb', True)
    #ax.hexbin(dens[selection],temp[selection], gridsize=250, bins='log', xscale='log', yscale='log', mincnt=1)
    ax.scatter(dens[selection],temp[selection], **kwargs)
    ax.set_xscale('log')
    ax.set_yscale('log')
    if cmb:
        ax.axhline(2.725 * (snap.header.Redshift + 1),
                   linestyle='--', color='k')
    return ax
    
clist = ['k', 'b', 'g', 'r', 'orange']
markCMB = [False, False, False, True, False]
#with sns.axes_style("white"):
fig, axarr = plt.subplots(4,1, sharex=True, sharey=True, figsize=(9,16))
for i,snap in enumerate(snaplist):
    for j in xrange(4):
        axarr[j] = plot_temp(axarr[j], snap, rls[j][i], cmb=markCMB[i], edgecolor=clist[i], s=3, c=clist[i])
        #axarr[j,i].plot(ndata.iat[j,i], Tdata.iat[j,i], 'ro', ms=8)
        #axarr[j].text(.03,.05, '{:.2f}'.format(mdata.iat[j,i]) + r'$\,$M$_{\odot}$', 
        #              ha='left', va='bottom', size=20, transform=axarr[j].transAxes)
        
axarr[0].set_xlim(1e5, 1.5e12)
axarr[0].set_ylim(50, 2e3)
for a in axarr[:]:
    a.set_ylabel('T [K]')
axarr[-1].set_xlabel('n [cm$^{-3}$]') 

plt.setp(axarr[-1].get_xticklabels()[1::2], visible=False)
fig.subplots_adjust(hspace=0, wspace=0)
'''
axarr[0,0].set_title(r'J = 0')
axarr[0,1].set_title(r'J = J$_{0}$')
axarr[0,2].set_title(r'J = 10 J$_{0}$')
axarr[0,3].set_title(r'J = 100 J$_{0}$')
'''
axarr[0].text(.03,.95, r'$\leq 1000\,$AU', ha='left', va='top', size=20, transform=axarr[0].transAxes)
axarr[1].text(.03,.95, r'$1000-2000\,$AU', ha='left', va='top', size=20, transform=axarr[1].transAxes)
axarr[2].text(.03,.95, r'$2000-3000\,$AU', ha='left', va='top', size=20, transform=axarr[2].transAxes)
axarr[3].text(.03,.95, r'$3000-4000\,$AU', ha='left', va='top', size=20, transform=axarr[3].transAxes)

fig.savefig('figures/radial_bins/temp-overlay.png', bbox_inches='tight')